In [2]:
import pandas as pd
data_dir='/home/rajesh/work/data/data_move_-1.csv'
df= pd.read_csv(data_dir)
selected_df=df[['date_x', 'Animal_ID', 'Event', 'THI', 'yield', 'water_intake in l', 'Days_in_Milk', 'rum_index', 'Group_ID']]
selected_df=selected_df.rename(columns={
    'date_x': 'date',
    'Animal_ID': 'cow_id',
    'Days_in_Milk': 'DIM',
    'yield': 'Milk',
    'rum_index': 'RT',
    'water_intake in l': 'Water',
    'THI': 'THI',
    'Event': 'events',
    'Group_ID': 'sensor_group'
})

/tmp/ipykernel_88001/4278121591.py:3: DtypeWarning: Columns (22,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(data_dir)


In [6]:
df.animal_id.nunique()

1130

In [7]:
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'date_x',
       'timestamp_x', 'pH', 'temperature °C', 'activity',
       'Temp Without Drinkcycles', 'Normal temperature', 'Heat index',
       'rum_index', 'animal_id', 'time', 'day', 'date_y', 'timestamp_y',
       'water_intake', 'date', 'Animal_ID', 'Group_ID', 'Date', 'Days_in_Milk',
       'Age_Days', 'Lactation_Num', 'Yield(gr)', 'Conductivity',
       'Milking_Time(seconds)', 'Event', 'THI', 'THI_class', 'seasons',
       'yield_cat', 'Event_new', 'day_event', 'date_relative',
       'water_intake in l', 'moved_days', 'yield', 'lac_stage'],
      dtype='object')